In [1]:
!sudo pip3 install requests_oauthlib

The directory '/home/ubuntu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/ubuntu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [18]:
#Objective - Store twitter trending topics in NYC and San Francisco over time in a MySQL database 
#MySQL Database IP Address - 34.235.205.203, and port 3306. So it can be connected at 34.235.205.203:3306
#Database Name - TwitterTrending
#Table Name - Trends
#Primary Key - (trending topic's url, woeid)
#Cron job scheduled to run every 30 minutes

import json
from datetime import datetime
from pprint import pprint

import requests
import requests_oauthlib
import MySQLdb as mdb
import re

#Keys used to access the Twitter API via the requests_oauthlib library
class TwitterConstants:
    consumer_key = "W7AAQByycHyFX2uT6kBtreJp2"
    consumer_secret = "0PiKkDxgHc2rw05usn7doPZZsTsdMaSuaXhXw7vdebFKQIEYkj"
    access_token_key = "900384218596954112-EoxlVD6IS90QaY6xbI3mJDvo7NtoI0H"
    access_token_secret = "c3a0eani09hfI5rNTm80JTxfuLoK5kZSOzNZJOeLzGGvS"


class TwitterTrending:
    #initializer for the class that takes in a physical address and makes a connection to the MySQL database
    def __init__(self, location_address="1 E 2nd Street, New York"):
        self.location_address = location_address
        self.connection = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True)
    #closes the connection after the object of the class is no longer used    
    def __del__(self):
        self.connection.close()

    #converts the physical address to a latitude and longitudes using the Google Maps API
    #uses the latitude and longitude value to find out the nearest woeid using Twitter's closes trends API
    #obtains the required woeid from the previous step and uses that to get the current trending topics.
    def get_trending_topics(self):
        url = 'https://maps.googleapis.com/maps/api/geocode/json'
        params = {'sensor': 'false', 'address': self.location_address}
        r = requests.get(url, params=params)
        results = r.json()['results']
        location = results[0]['geometry']['location']

        auth = requests_oauthlib.OAuth1(TwitterConstants.consumer_key,
                                        TwitterConstants.consumer_secret,
                                        TwitterConstants.access_token_key,
                                        TwitterConstants.access_token_secret)

        trending_topics_url_woeid = "https://api.twitter.com/1.1/trends/closest.json?lat={}&long={}".format(
            location['lat']
            ,
            location['lng'])
        response = requests.get(trending_topics_url_woeid, auth=auth)
        parsed_json = json.loads(response.text)
        woe_id = parsed_json[0].get("woeid")
        url = parsed_json[0].get("url")

        trending_topics_url = "https://api.twitter.com/1.1/trends/place.json?id={}".format(woe_id)
        response = requests.get(trending_topics_url, auth=auth)
        response_json = json.loads(response.text)
        
      
        common_values = {}
        twitter_time_string = response_json[0].get("as_of")
        
        datetime_list = str(twitter_time_string).split("T")
        date = datetime_list[0]
        time = datetime_list[1].split("Z")[0]

        date_time_str = datetime.strptime((date + " " + time), "%Y-%m-%d %H:%M:%S")
        common_values["time"] = date_time_str

        common_values["woeid"] = woe_id
        common_values["location"] = str(location['lat']) + ":" + str(location['lng'])

        return_list = []

        for value in response_json[0].get("trends"):
            value["time"] = common_values.get("time")
            value["woeid"] = common_values.get("woeid")
            value["location"] = common_values.get("location")

            return_list.append(value)

        pprint(return_list)

        return return_list
    
    #Creates database when required
    def create_db(self):
        create_db_query = "CREATE DATABASE IF NOT EXISTS TwitterTrending DEFAULT CHARACTER SET 'utf8'"
        cursor = self.connection.cursor()
        cursor.execute(create_db_query)
        cursor.close()
        
    #Creates table when required    
    def create_table(self):
        create_table_query = '''CREATE TABLE IF NOT EXISTS TwitterTrending.Trends (location TEXT, name TEXT, promoted_content TEXT, query TEXT, time DATETIME, tweet_volume INT, url VARCHAR(255), woeid INT, PRIMARY KEY(url,woeid));'''
        print(create_table_query)
        cursor = self.connection.cursor()
        cursor.execute(create_table_query)
        cursor.close()
    
    #Inserts the data obtained from get_trending_topics() into the database
    def insert_into_db(self):
        self.create_db()
        self.create_table()
        trending_topics = self.get_trending_topics()
        cursor = self.connection.cursor()
        query = "INSERT IGNORE INTO TwitterTrending.Trends VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
        print(query)
        for trend in trending_topics:
            #handling some important cases where tweet_volume may not be present.
            tweet_volume = trend.get("tweet_volume")
            if tweet_volume == None:
                tweet_volume = 0
            
            name = trend.get("name","NULL")
            
            #using re to escape problematic characters that can negatively impact the proper insertion of 
            #values into the MySQL database.
            name = re.escape(name)

           
            query_parameters = (trend.get('location'),name,trend.get("promoted_content"),trend.get("query"),
                                trend.get("time"),tweet_volume,trend.get("url"),trend.get("woeid"))


            
            cursor.execute(query,query_parameters)
            print(query_parameters)
            
        self.connection.commit()
       
        
        cursor.close()
            
    
    


if __name__ == "__main__":
    twitter = TwitterTrending()
    twitter.insert_into_db()
    
    twitter_California = TwitterTrending("San Franscisco, California")
    twitter_California.insert_into_db()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: Warning: (1007, "Can't create database 'TwitterTrending'; database exists")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:104: Warning: (1050, "Table 'Trends' already exists")


CREATE TABLE IF NOT EXISTS TwitterTrending.Trends (location TEXT, name TEXT, promoted_content TEXT, query TEXT, time DATETIME, tweet_volume INT, url VARCHAR(255), woeid INT, PRIMARY KEY(url,woeid));
[{'location': '40.7254116:-73.99190089999999',
  'name': '#Yankees',
  'promoted_content': None,
  'query': '%23Yankees',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 14),
  'tweet_volume': 39817,
  'url': 'http://twitter.com/search?q=%23Yankees',
  'woeid': 2459115},
 {'location': '40.7254116:-73.99190089999999',
  'name': '#ThisIsUs',
  'promoted_content': None,
  'query': '%23ThisIsUs',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 14),
  'tweet_volume': 29930,
  'url': 'http://twitter.com/search?q=%23ThisIsUs',
  'woeid': 2459115},
 {'location': '40.7254116:-73.99190089999999',
  'name': '#AHSCult',
  'promoted_content': None,
  'query': '%23AHSCult',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 14),
  'tweet_volume': 38361,
  'url': 'http://twitter.com/search?q=%23AHSCult',
  '

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23Yankees-2459115' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23ThisIsUs-2459115' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23NationalBoyfriendDay-2459115' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23LasVegasShooting-2459115' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%22Tom+Petty%22-2459115' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%22G

[{'location': '37.7749295:-122.4194155',
  'name': '#NationalBoyfriendDay',
  'promoted_content': None,
  'query': '%23NationalBoyfriendDay',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 15),
  'tweet_volume': 248172,
  'url': 'http://twitter.com/search?q=%23NationalBoyfriendDay',
  'woeid': 2487956},
 {'location': '37.7749295:-122.4194155',
  'name': 'Twins',
  'promoted_content': None,
  'query': 'Twins',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 15),
  'tweet_volume': 135654,
  'url': 'http://twitter.com/search?q=Twins',
  'woeid': 2487956},
 {'location': '37.7749295:-122.4194155',
  'name': 'Yahoo',
  'promoted_content': None,
  'query': 'Yahoo',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 15),
  'tweet_volume': 328868,
  'url': 'http://twitter.com/search?q=Yahoo',
  'woeid': 2487956},
 {'location': '37.7749295:-122.4194155',
  'name': 'Tom Petty',
  'promoted_content': None,
  'query': '%22Tom+Petty%22',
  'time': datetime.datetime(2017, 10, 4, 3, 19, 15),
  'tweet_vo

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23NationalBoyfriendDay-2487956' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=Twins-2487956' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=Yahoo-2487956' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%22Tom+Petty%22-2487956' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23GHC17-2487956' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:130: Warning: (1062, "Duplicate entry 'http://twitter.com/search?q=%23VFSummit-2487956' for k